# PPR 

<img src="{{site.baseurl}}/images/ppr4r.png" width = "100%">



# List being Created:

    async function loadAttempts() {
    const currentUserResponse = await fetch(${pythonURI}/api/id, fetchOptions);
    if (!currentUserResponse.ok) throw new Error('Failed to fetch current user');
    const currentUser = await currentUserResponse.json();
    userName = currentUser.uid;
    userID = currentUser.id;

    const quizGrading = await fetch(quizGradingsApi, fetchOptions);
    if (!quizGrading.ok) {console.error("Error loading attempts:", quizGrading);}

    const quizResults = await quizGrading.json();
    console.log(quizResults);
    }

**Explanation:** The list quizResults is created by fetching the quiz attempts and scores from the backend and storing it in a variable to be processed and then displayed and loaded on the table. 

# List being processed:

    quizResults.forEach(attempt => {
    const row = document.createElement('tr');
    const idCell = document.createElement('td');
    idCell.innerHTML = attempt.id;
    const usernameCell = document.createElement('td');
    usernameCell.innerHTML = attempt.username;
    const quizgradeCell = document.createElement('td');
    quizgradeCell.innerHTML = attempt.quizgrade;
    const attemptCell = document.createElement('td');
    attemptCell.innerHTML = attempt.attempt;
    const actionCell = document.createElement('td');
    
    if (attempt.username === userName) {
        const deleteButton = document.createElement('button');
        deleteButton.innerHTML = 'Delete';
        deleteButton.addEventListener('click', () => deleteAttempt(attempt.id));
        const editButton = document.createElement('button');
        editButton.innerHTML = 'Edit';
        editButton.addEventListener('click', () => editAttempt(attempt.id));
        actionCell.append(deleteButton);
        actionCell.append(editButton);
    }
    
    row.append(idCell);
    row.append(usernameCell);
    row.append(quizgradeCell);
    row.append(attemptCell);
    row.append(actionCell);
    tbody.append(row);
    });

**Explanation:** After fetching the quizResults, it's processed by iterating through the list and updating the DOM (displaying the attempts in the table). The quizResults array is processed by iterating over each attempt, creating a new table row for each one, and then displaying the data inside the row.

# A Function:

    async function loadAttempts() {
    }

**Explanation:** The function responsible for loading and processing the quiz attempt data is loadAttempts. It fetches data and populates the table with quiz scores and attempts.

# Call to Function:

    window.onload = () => {
    const selectedQuestions = randomizeQuestions(Questions, 5);
    buildQuiz(selectedQuestions);
    loadAttempts(); 
    };

**Explanation:** The loadAttempts function is called and ensures that the quiz attempts and scores are displayed in the table after 